**NOTE**: Try to solve this on your own just from the PDF definition of the problem before looking at this solution!

# Retirement Model

This is a retirement model which models salary with a constant growth rate, and savings rate changes to three different levels depending on the level of the salary. The model is divided into the following sections:
- [**Setup**](#Setup): Runs any imports and other setup
- [**Inputs**](#Inputs): Defines the inputs for the model
- [**Salaries**](#Salaries): Determining the salary in each year, considering constant salary growth
- [**Wealths**](#Wealths): Determining the wealth in each year, considering an investment rate and a savings rate which is dependent on the level of salary.
- [**Retirement**](#Retirement): Determines years to retirement from the wealths over time, the main output from the model.

## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [2]:
from dataclasses import dataclass

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [3]:
@dataclass
class ModelInputs:
    starting_salary: int = 50000
    salary_growth: float = 0.03
    mid_salary_cutoff: int = 80000
    high_salary_cutoff: int = 120000
    low_savings_rate: float = 0.1
    mid_savings_rate: float = 0.25
    high_savings_rate: float = 0.4
    interest_rate: float = 0.05
    desired_cash: int = 1500000
        
model_data = ModelInputs()
model_data

ModelInputs(starting_salary=50000, salary_growth=0.03, mid_salary_cutoff=80000, high_salary_cutoff=120000, low_savings_rate=0.1, mid_savings_rate=0.25, high_savings_rate=0.4, interest_rate=0.05, desired_cash=1500000)

## Salaries

Here the salary for each year is calculated. We assume that the salary grows at a constant rate each year. Based on this assumption, the salary would evolve over time with the following equation:

$$s_t = s_0 (1 + g_s)^n$$

Where:
- $s_t$: Salary at year $t$
- $s_0$: Starting salary (year 0)
- $g_s$: Salary growth
- $n$: Number of years

And in Python format:

In [4]:
def salary_at_year(data: ModelInputs, year):
    """
    Gets the salary at a given year from the start of the model based on constant salary growth.
    """
    # This is the formula above implemented in Python
    salary_t = data.starting_salary * (1 + data.salary_growth) ** year
    return salary_t

That function will get the salary at a given year, so to get all the salaries we just run it on each year. But we will not know how many years to run as we should run it until the individual is able to retire. So we are just showing the first few salaries for now and will later use this function in the [Wealths](#Wealths) section of the model.

In [13]:
for i in range(20):
    year = i + 1
    salary = salary_at_year(model_data, year)
    print(f'The salary at year {year} is ${salary:,.0f}.')

The salary at year 1 is $51,500.
The salary at year 2 is $53,045.
The salary at year 3 is $54,636.
The salary at year 4 is $56,275.
The salary at year 5 is $57,964.
The salary at year 6 is $59,703.
The salary at year 7 is $61,494.
The salary at year 8 is $63,339.
The salary at year 9 is $65,239.
The salary at year 10 is $67,196.
The salary at year 11 is $69,212.
The salary at year 12 is $71,288.
The salary at year 13 is $73,427.
The salary at year 14 is $75,629.
The salary at year 15 is $77,898.
The salary at year 16 is $80,235.
The salary at year 17 is $82,642.
The salary at year 18 is $85,122.
The salary at year 19 is $87,675.
The salary at year 20 is $90,306.


As expected, with the default inputs, the salary is increasing at 3% per year. Once we go to the [Wealths](#Wealths) portion of the model, we should see additional savings happen in year 16 due to an increased savings rate.

## Wealths

The wealths portion of the model is concerned with applying the savings rate to the earned salary to calculate the cash saved, accumulating the cash saved over time, and applying the investment rate to the accumulated wealth. The savings rate needs to be determined by determining whether the current salary is low, medium, or high, based off the cutoff inputs.

The savings rate can be represented by the following stepwise function:

$$r_{st} = \begin{cases} 
      r_{sl} & \text{if } s_t\leq c_m \\
      r_{sm} & \text{if } c_m \leq s_t\leq c_h \\
      r_{sh} & \text{if } s_t \geq c_h 
   \end{cases}
$$
Where:
- $r_{st}$: Savings rate at year $t$
- $r_{sl}$: Savings rate for low salaries
- $r_{sm}$: Savings rate for mid salaries
- $r_{sh}$: Savings rate for high salaries
- $c_m$: Mid-salary cutoff
- $c_h$: High-salary cutoff


In Python, this is a simple `if/elif/else` structure:

In [19]:
def get_savings_rate(data: ModelInputs, salary):
    if salary < data.mid_salary_cutoff:
        return data.low_savings_rate
    elif salary < data.high_salary_cutoff:
        return data.mid_savings_rate
    else:
        return data.high_savings_rate

To calculate cash saved, it is simply:

$$c_t = s_t * r_{st}$$

Where:
- $c_t$: Cash saved during year $t$

In [14]:
def cash_saved_during_year(data: ModelInputs, year):
    """
    Calculated the cash saved within a given year, by first calculating the salary at that year,
    then determining the savings rate for that level of salary, then applying the savings rate to the salary.
    """
    salary = salary_at_year(data, year)
    savings_rate = get_savings_rate(data, salary)
    cash_saved = salary * savings_rate
    return cash_saved

To get the wealth at each year, it is just applying the investment return to last year's wealth, then adding this year's cash saved:

$$w_t = w_{t-1} (1 + r_i) + c_t$$
Where:
- $w_t$: Wealth at year $t$
- $r_i$: Investment rate

In [15]:
def wealth_at_year(data: ModelInputs, year, prior_wealth):
    """
    Calculate the accumulated wealth for a given year, based on previous wealth, the investment rate,
    and cash saved during the year.
    """
    cash_saved = cash_saved_during_year(data, year)
    wealth = prior_wealth * (1 + data.interest_rate) + cash_saved
    return wealth

Again, just like in the [Salaries](#Salaries) section, we can now get the output for each year, but we don't know ultimately how many years we will have to run it. That will be determined in the [Retirement](#Retirement) section. So for now, just show the first few years of wealth accumulation:

In [20]:
prior_wealth = 0  # starting with no cash saved
for i in range(20):
    year = i + 1
    wealth = wealth_at_year(model_data, year, prior_wealth)
    print(f'The wealth at year {year} is ${wealth:,.0f}.')
    
    # Set next year's prior wealth to this year's wealth
    prior_wealth = wealth

The wealth at year 1 is $5,150.
The wealth at year 2 is $10,712.
The wealth at year 3 is $16,711.
The wealth at year 4 is $23,174.
The wealth at year 5 is $30,129.
The wealth at year 6 is $37,606.
The wealth at year 7 is $45,636.
The wealth at year 8 is $54,251.
The wealth at year 9 is $63,488.
The wealth at year 10 is $73,382.
The wealth at year 11 is $83,972.
The wealth at year 12 is $95,300.
The wealth at year 13 is $107,407.
The wealth at year 14 is $120,341.
The wealth at year 15 is $134,147.
The wealth at year 16 is $160,914.
The wealth at year 17 is $189,620.
The wealth at year 18 is $220,381.
The wealth at year 19 is $253,319.
The wealth at year 20 is $288,562.


With default inputs, the wealth is going up by approximately 10% of the salary each year, plus a bit more for investment. As we saw from the [Salaries](#Salaries) section, with the default inputs, in year 16 we hit the mid-salary cutoff for a higher savings rate. And indeed here, once we hit year 16, a substantially larger increase in wealth begins to be recorded.

## Retirement

This section of the model puts everything together to produce the final output of years to retirement. It uses the logic to get the wealths at each year, which in turn uses the logic to the get salary at each year. The wealth at each year is tracked over time until it hits the desired cash. Once the wealth hits the desired cash, the individual is able to retire so that year is returned as the years to retirement.

In [21]:
def years_to_retirement(data: ModelInputs):
    
    # starting with no cash saved
    prior_wealth = 0  
    wealth = 0
    
    year = 0  # will become 1 on first loop
    
    print('Wealths over time:') # \n makes a blank line in the output.
    while wealth < data.desired_cash:
        year = year + 1
        wealth = wealth_at_year(model_data, year, prior_wealth)
        print(f'The wealth at year {year} is ${wealth:,.0f}.')

        # Set next year's prior wealth to this year's wealth
        prior_wealth = wealth
        
    # Now we have exited the while loop, so wealth must be >= desired_cash. Whatever last year was set
    # is the years to retirement.
    print(f'\nRetirement:\nIt will take {year} years to retire.')  # \n makes a blank line in the output.
    return year

With the default inputs:

In [22]:
years = years_to_retirement(model_data)

Wealths over time:
The wealth at year 1 is $5,150.
The wealth at year 2 is $10,712.
The wealth at year 3 is $16,711.
The wealth at year 4 is $23,174.
The wealth at year 5 is $30,129.
The wealth at year 6 is $37,606.
The wealth at year 7 is $45,636.
The wealth at year 8 is $54,251.
The wealth at year 9 is $63,488.
The wealth at year 10 is $73,382.
The wealth at year 11 is $83,972.
The wealth at year 12 is $95,300.
The wealth at year 13 is $107,407.
The wealth at year 14 is $120,341.
The wealth at year 15 is $134,147.
The wealth at year 16 is $160,914.
The wealth at year 17 is $189,620.
The wealth at year 18 is $220,381.
The wealth at year 19 is $253,319.
The wealth at year 20 is $288,562.
The wealth at year 21 is $326,243.
The wealth at year 22 is $366,507.
The wealth at year 23 is $409,502.
The wealth at year 24 is $455,387.
The wealth at year 25 is $504,328.
The wealth at year 26 is $556,502.
The wealth at year 27 is $612,094.
The wealth at year 28 is $671,297.
The wealth at year 29 i